In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AdamW
import torch

In [2]:
import nltk
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
import pandas as pd
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [5]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
from nltk.corpus import stopwords
sw = stopwords.words('english')

def clean_text(text):

    text = text.lower()

    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    text = re.sub(r"http\S+", "",text)
    html=re.compile(r'<.*?>')
    text = html.sub(r'',text)

    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'')

    text = [word.lower() for word in text.split() if word.lower() not in sw]

    text = " ".join(text)
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

In [7]:
df['text'] = df['text'].apply(clean_text)

In [8]:
df = df.drop(columns=['id','keyword','location'], axis=1)

In [9]:
df

,text,target
0,deeds reason earthquake may allah forgive us,1
1,forest fire near la ronge sask canada,1
2,residents asked shelter place notified officer...,1
3,", people receive wildfires evacuation orders c...",1
4,got sent photo ruby alaska smoke wildfires pou...,1
...,...,...
7608,two giant cranes holding bridge collapse nearb...,1
7609,aria ahrary thetawniest control wild fires cal...,1
7610,utc km volcano hawaii http tco zdtoyd ebj,1
7611,police investigating e bike collided car littl...,1


In [10]:
!pip install -q datasets

In [11]:
from datasets import Dataset
dataset = Dataset.from_pandas(df[['text', 'target']])

In [12]:
dataset

Dataset({
    features: ['text', 'target'],
    num_rows: 7613
})

In [13]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
max_length=0
for tweet in dataset['text']:
    length = len(tweet)
    if length > max_length:
        max_length = length
print(length)

74


In [15]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        pad_to_max_length=True,
        padding='max_length',
        truncation=True,
        max_length=45,
        return_token_type_ids=False,
        return_attention_mask=True,
        return_tensors='pt'
    )
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7613 [00:00<?, ? examples/s]

In [16]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

In [17]:
train_dataset = train_dataset.rename_column("target", "labels")
val_dataset = val_dataset.rename_column("target", "labels")

In [18]:
train_dataset

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 6090
})

In [19]:
val_dataset

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 1523
})

In [20]:
train_dataset = train_dataset.remove_columns(['text'])
val_dataset = val_dataset.remove_columns(['text'])

In [21]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [23]:
def model_init(trial):
    return BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, output_attentions=False,
    output_hidden_states=False)

In [24]:
training_args = TrainingArguments(
    output_dir='./results',
    warmup_steps=500,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to='none'
)

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1, 
        'precision': precision,
        'recall': recall
    }


In [26]:
trainer = Trainer(
    model=None,
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
!pip install optuna

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
def optuna_hp_space(trial):
    return {
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 5e-5, log=True),
        'per_device_train_batch_size': trial.suggest_categorical('per_device_train_batch_size', [16, 32]),
        'per_device_eval_batch_size': trial.suggest_categorical('per_device_eval_batch_size', [16, 32]),
        'num_train_epochs': trial.suggest_int('num_train_epochs', 2, 4),
        'weight_decay': trial.suggest_float('weight_decay', 0.1, 0.3)
    }

In [29]:
def compute_objective(metrics):
    return metrics["eval_accuracy"] 

In [30]:
best_trials = trainer.hyperparameter_search(
    direction='maximize',
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=5,
    compute_objective=compute_objective,
)

[I 2024-10-08 17:01:15,753] A new study created in memory with name: no-name-dfb24c6f-c026-4883-9f32-45212613199b
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.634500,0.525384,0.751149,0.726748,0.687585,0.770642
200,0.462800,0.442042,0.804334,0.769706,0.778125,0.761468
300,0.406300,0.436850,0.820749,0.779305,0.826758,0.737003
400,0.379400,0.464576,0.822718,0.780488,0.833333,0.733945
500,0.323400,0.428270,0.820092,0.783570,0.810458,0.758410
600,0.327700,0.591030,0.816809,0.764954,0.851782,0.694190
700,0.218200,0.495416,0.822062,0.783026,0.821849,0.747706


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.649100,0.557748,0.732108,0.706897,0.666667,0.752294
200,0.480600,0.462656,0.794485,0.766940,0.747460,0.787462
300,0.412600,0.421935,0.826658,0.782537,0.848214,0.726300
400,0.379300,0.450321,0.820092,0.775777,0.834507,0.724771
500,0.328400,0.430183,0.826001,0.786118,0.832479,0.744648
600,0.327500,0.511029,0.820749,0.776046,0.837168,0.723242
700,0.224500,0.509201,0.813526,0.773885,0.807309,0.743119


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

...

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.651700,0.554487,0.740643,0.697318,0.698925,0.695719
200,0.489800,0.447127,0.803677,0.762887,0.792422,0.735474


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.629100,0.522670,0.756402,0.727406,0.700141,0.756881
200,0.463500,0.443832,0.806303,0.772902,0.778295,0.767584
300,0.407600,0.424560,0.826658,0.776650,0.869318,0.701835
400,0.379400,0.463897,0.821405,0.772194,0.853704,0.704893
500,0.321600,0.433273,0.822062,0.781275,0.827350,0.740061


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

In [31]:
best_params = best_trials.hyperparameters
print(best_params)

{'learning_rate': 4.171479503407097e-05, 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 32, 'num_train_epochs': 4, 'weight_decay': 0.1395962005551185}


In [32]:
training_args = TrainingArguments(
    output_dir='./model',
    warmup_steps=500,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to='none',
    **best_params )

trainer = Trainer(
    model=model_init(best_trials),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
trainer.train()

trainer.save_model('./best_model')

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.649600,0.494417,0.773473,0.726841,0.753695,0.701835
200,0.454100,0.439334,0.812869,0.775059,0.800979,0.750765
300,0.412000,0.411703,0.817466,0.772131,0.832155,0.720183
400,0.374400,0.455324,0.820749,0.775309,0.839572,0.720183
500,0.319800,0.434426,0.818779,0.766102,0.859316,0.691131
600,0.324000,0.535977,0.815496,0.768724,0.832442,0.714067
700,0.215700,0.547293,0.819435,0.776241,0.829565,0.729358


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

In [35]:
best_model = BertForSequenceClassification.from_pretrained('./best_model')

In [36]:
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [37]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [38]:
test_df['text'] = test_df['text'].apply(clean_text)

In [39]:
test_df = test_df.drop(columns=['keyword','location'], axis=1)
test_df

,id,text
0,0,happened terrible car crash
1,2,"heard earthquake different cities, stay safe e..."
2,3,"forest fire spot pond, geese fleeing across st..."
3,9,apocalypse lighting spokane wildfires
4,11,typhoon soudelor kills china taiwan
...,...,...
3258,10861,earthquake safety los angeles safety fasteners...
3259,10865,storm ri worse last hurricane city amp others ...
3260,10868,green line derailment chicago http tco utbxlcbiuy
3261,10874,meg issues hazardous weather outlook hwo http ...


In [67]:
text = test_df['text'].values

In [68]:
test_input_ids = []
test_attention_masks = []
for tweet in text:
    encoded_dict =tokenizer(tweet,
                add_special_tokens = True,
                return_tensors='pt',
                return_attention_mask=True,
                max_length=45,
                pad_to_max_length=True
                )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2837: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [79]:
test_input_ids

tensor([[  101,  3047,  6659,  ...,     0,     0,     0],
        [  101,  2657,  8372,  ...,     0,     0,     0],
        [  101,  3224,  2543,  ...,     0,     0,     0],
        ...,
        [  101,  2665,  2240,  ...,     0,     0,     0],
        [  101, 12669,  3314,  ...,     0,     0,     0],
        [  101,  2103, 11253,  ...,     0,     0,     0]])

In [80]:
from torch.utils.data import DataLoader, TensorDataset, SequentialSampler
test_dataset = TensorDataset(test_input_ids,test_attention_masks)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset),batch_size=32)

In [84]:
import numpy as np
predictions = []
for index,batch in enumerate(test_dataloader):
    b_input_ids = batch[0]
    b_input_mask = batch[1]
    with torch.no_grad():
        output = best_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = output.logits
        logits = logits.detach().numpy()
        pred_flat = np.argmax(logits, axis=1).flatten()
        predictions.extend(list(pred_flat))

In [89]:
df = pd.DataFrame()
df['id'] = test_df['id']
df['target'] = predictions
df.to_csv('submission.csv',index=False)